# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
#print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION =
    {'class': 'SimpleStrategy', 'replication_factor': 1}"""
)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following questions of the data:

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Question 1 expects the name of the artist, title of the song, and length of the song based on a given sessionId and and itemInSession. Thus, the SELECT statement will look like:
```
SELECT artistName, song, length FROM table WHERE sessionId=value AND itemInSession=value
```
After knowing the query we wish to run, we can properly model the data and create the table.

- TABLE NAME: the name of the table should be relevant to the queries that will be run on it, in this case the artist, song, and length of song listened to in a given session, so it will be named 'song_session'.
- PRIMARY KEY: we will need to uniquely identify each row in the table, else data could be overwritten. Since we need to return results based on both sessionID and itemInSession we will need to set both as the PRIMARY KEY.
- COLUMN NAMES: we need to return the name of the artist, title of the song, and length of the song based on sessionId and itemInSession so we will need to create columns for each. 

In [8]:
query = "CREATE TABLE IF NOT EXISTS song_session "
query = query + "(sessionId int, itemInSession int, artistName text, song text, length float, \
        PRIMARY KEY (sessionId, itemInSession))"
session.execute(query)

In [9]:
# Code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_session (sessionId, itemInSession, artistName, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query = "SELECT artistName, song, length FROM song_session WHERE sessionId=338 AND itemInSession=4"
rows = session.execute(query)
for row in rows:
    print(row)

Row(artistname='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Question 2 expects the name of the artist, title of the song, and the user's first and last name based on a given userId and sessionId. Thus, the SELECT statement will look like:
```
SELECT artistName, song, firstName, lastName FROM table WHERE userId=value AND sessionId=value
```
After knowing the query we wish to run, we can properly model the data and create the table.

- TABLE NAME: the name of the table should be relevant to the queries that will be run on it, in this case the names of the artists and songs a user has listened to in a session, so it will be named 'user_session'.
- PRIMARY KEY: we will need to uniquely identify each row in the table, else data could be overwritten. Since we need to return results based on both userId and sessionId we will need to set both as the PRIMARY KEY. We also want the results sorted by itemInSession so we need to add that to the PRIMARY KEY as a CLUSTERING COLUMN.
- COLUMN NAMES: we need to return the name of the artist, title of the song, and the user's first and last name based on userId and sessionId so we will need to create columns for each. We also want the data sorted by itemInSession, so we will need to add that column as well.

In [11]:
query = "CREATE TABLE IF NOT EXISTS user_session "
query = query + "(userId int, sessionId int, itemInSession int, artistName text, song text, firstName text, \
        lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
session.execute(query)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_session (userId, sessionId, itemInSession, firstName, lastName, \
                artistName, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
query = "SELECT artistName, song, firstName, lastName FROM user_session WHERE userId=10 AND sessionId=182"
rows = session.execute(query)
for row in rows:
    print(row)

Row(artistname='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artistname='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artistname='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artistname='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Question 3 expects the user's first and last based on a given song title. Thus, the SELECT statement will look like:
```
SELECT firstName, lastName FROM table WHERE song=value AND
```
After knowing the query we wish to run, we can properly model the data and create the table.

- TABLE NAME: the name of the table should be relevant to the queries that will be run on it, in this case the names of the user's who've listened to a given song, so it will be named 'song_listeners'.
- PRIMARY KEY: we will need to uniquely identify each row in the table, else data could be overwritten. We need to return the user who's listened to a specific song, so we will need both the song title and userId as the PRIMARY KEY.
- COLUMN NAMES: we need to return the user's first and last name based on the song title so we will need to create columns for each. We also need to retrieve the user's based on their userId, since users could have the same name, so we need a column for userId as well.

In [14]:
query = "CREATE TABLE IF NOT EXISTS song_listeners "
query = query + "(song text, userId int, firstName text, lastName text, \
        PRIMARY KEY (song, userId))"
session.execute(query)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_listeners (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [16]:
query = "SELECT firstName, lastName FROM song_listeners WHERE song='All Hands Against His Own'"
rows = session.execute(query)
for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions
query = "DROP TABLE song_session"
session.execute(query)

query = "DROP TABLE user_session"
session.execute(query)

query = "DROP TABLE song_listeners"
session.execute(query)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()